In [1]:
from nat_zarcos import lattice

In [2]:
lat0 = lattice()
lat0.get_lattice()
lat0.coordinates

nothing to get: lattice folder not defined


array([[0.33333333, 0.33333333]])

In [3]:
lat1 = lattice('/home/akandra/O_Pt111/zacros_calculations/fn_3leed/jobs/1/traj_1')
lat1.site_coordinates

array([[  1.4105675 ,   0.81439153],
       [  2.821135  ,   3.2575661 ],
       [  4.2317025 ,   5.70074068],
       ...,
       [163.62583   ,  91.21185089],
       [165.0363975 ,  93.65502547],
       [166.446965  ,  96.09820005]], shape=(1600, 2))